In [1]:
from stratopy import metadatatools
from stratopy.extractors import cloudsat
from stratopy.extractors import goes
from stratopy.transformers import mergers
from stratopy.transformers import scalers

Download 1 band file from ABI GOES16

In [2]:
goes_obj = goes.GOES16(product_type="ABI-L2-CMIPF", channel=13)
goes_data = goes_obj.fetch("2019 jan 2 19:30")

In [3]:
goes_data.time_coverage_start,goes_data.time_coverage_end

('2019-01-02T19:00:36.3Z', '2019-01-02T19:11:14.1Z')

Lets see one transformer. This one normalizes an ABI image to [0,1].

Returns an xarray with an image of shape (n_bands, image_width, image_height).

In [3]:
norm_goes = scalers.MinMaxNormalize().transform(goes_data)
norm_goes

<xarray.Dataset>
Dimensions:                                 (y: 5424, x: 5424, nbands: 1,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates:
  * y                                       (y) float32 0.1518 ... -0.1518
  * x                                       (x) float32 -0.1518 ... 0.1518
  * nbands                                  (nbands) int64 1
    t                                       datetime64[ns] ...
    y_image                                 float32 ...
    x_image                                 float32 ...
    band_wavelength                         (band) float32 ...
    band_id                                 (band) int32 ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/29)
    CMI                                     (nbands, y, x) float32 nan ... nan
    DQF                                     (y, x) float32 ...
    time_bounds                             (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                  int32 ...
    y_image_bounds                          (number_of_image_bounds) float32 ...
    x_image_bounds                          (number_of_image_bounds) float32 ...
    ...                                      ...
    algorithm_dynamic_input_data_container  int32 ...
    percent_uncorrectable_GRB_errors        float32 ...
    percent_uncorrectable_L0_errors         float32 ...
    earth_sun_distance_anomaly_in_AU        float32 ...
    processing_parm_version_container       int32 ...
    algorithm_product_version_container     int32 ...
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v25, 05 July 2013)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    time_coverage_start:       2019-01-02T18:00:36.3Z
    time_coverage_end:         2019-01-02T18:11:14.1Z
    timeline_id:               ABI Mode 3
    production_data_source:    Realtime
    id:                        3ad0c10b-5583-45dd-a158-199addc5b811
    _STRATOPY_:                Metadata(orbit_type='geostationary', platform=...

Download a file from CloudSat.

In [4]:
from unittest import mock

with mock.patch("paramiko.RSAKey.from_private_key_file", return_value="pkey"):
    with mock.patch("paramiko.SSHClient.connect"):
        cs_obj = cloudsat.CloudSat(
        product_type="2B-CLDCLASS.P1_R05",
        username="fakeusr@gmail.edu",
        keyfile="some_file",
        )
        
        csat_data = cs_obj.fetch("2019 dec 12 15:00")

AttributeError: 'NoneType' object has no attribute 'open_sftp_client'

In [5]:
CLOUDSAT_PATH = "../../../tests/data/CloudSat/2019002175851_67551_CS_2B-CLDCLASS_GRANULE_P1_R05_E08_F03.hdf"
csat_data = cloudsat.read_hdf4(CLOUDSAT_PATH)
csat_data = metadatatools.add_metadata(
        csat_data,
        orbit_type=metadatatools.POLAR,
        platform=metadatatools.CLOUDSAT,
        instrument_type=metadatatools.RADARS,
        product_key="2B-CLDCLASS.P1_R05",
    )

Merge products

In [6]:
merged_data = mergers.MergePolarGeos("2019 2 jan 18:30", 
                                     time_zone="UTC", 
                                     trim_size=(3,3)).transform(csat_data, norm_goes)

In [7]:
merged_data

<xarray.Dataset>
Dimensions:           (cloudsat_trace: 36950, z: 125, layer: 10, nbands: 1,
                       img_wide: 3, img_height: 3)
Coordinates:
    trace             object <built-in method copy of numpy.ndarray object at...
    height            (cloudsat_trace, z) int16 -9999 -9999 ... -9999 -9999
    layers            object <built-in method copy of numpy.ndarray object at...
    lat               (cloudsat_trace) float64 -0.009365 -0.01907 ... 0.004176
    lon               (cloudsat_trace) float64 117.1 117.1 117.1 ... 92.5 92.5
    time              (cloudsat_trace) datetime64[ns] 2019-01-02T17:59:01.740...
    precip_flag       (cloudsat_trace) int8 9 9 9 9 9 9 9 9 ... 9 9 9 9 9 9 9 9
    land_sea_flag     (cloudsat_trace) float32 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0
  * cloudsat_trace    (cloudsat_trace) int32 0 1 2 3 ... 36946 36947 36948 36949
  * nbands            (nbands) int8 1
  * img_wide          (img_wide) int8 1 2 3
  * img_height        (img_height) int8 1 2 3
Dimensions without coordinates: z, layer
Data variables:
    cloud_scenario    (cloudsat_trace, z) int16 2081 2081 2081 ... 2113 2113
    cloud_layer_type  (cloudsat_trace, layer) int8 0 0 0 0 0 0 0 ... 0 0 0 0 0 0
    cloud_layer_base  (cloudsat_trace, layer) float32 -99.0 -99.0 ... -99.0
    cloud_layer_top   (cloudsat_trace, layer) float32 -99.0 -99.0 ... -99.0
    geos              (cloudsat_trace, nbands, img_wide, img_height) float32 ...
Attributes:
    TAIstart:             820605541.74
    UTCstart:             [[64731.73828125]]
    bin_size:             [239.82907104]
    time_coverage_start:  2019-01-02T17:59:01.740000+00:00
    time_coverage_end:    2019-01-02T19:37:33.579844+00:00
    platform_ID:          CloudSat
    _STRATOPY_:           Metadata(orbit_type='polar', platform='CloudSat', i...

In [11]:
norm_goes.time_coverage_start, norm_goes.time_coverage_end

('2019-01-02T18:00:36.3Z', '2019-01-02T18:11:14.1Z')

In [ ]:
norm_goes[]

In [10]:
merged_data.time_coverage_start, merged_data.time_coverage_end

('2019-01-02T17:59:01.740000+00:00', '2019-01-02T19:37:33.579844+00:00')

In [12]:
goes_data.attrs

{'naming_authority': 'gov.nesdis.noaa',
 'Conventions': 'CF-1.7',
 'Metadata_Conventions': 'Unidata Dataset Discovery v1.0',
 'standard_name_vocabulary': 'CF Standard Name Table (v25, 05 July 2013)',
 'institution': 'DOC/NOAA/NESDIS > U.S. Department of Commerce, National Oceanic and Atmospheric Administration, National Environmental Satellite, Data, and Information Services',
 'project': 'GOES',
 'production_site': 'NSOF',
 'production_environment': 'OE',
 'spatial_resolution': '2km at nadir',
 'orbital_slot': 'GOES-East',
 'platform_ID': 'G16',
 'instrument_type': 'GOES R Series Advanced Baseline Imager',
 'scene_id': 'Full Disk',
 'instrument_ID': 'FM1',
 'dataset_name': 'OR_ABI-L2-CMIPF-M3C13_G16_s20190021800363_e20190021811141_c20190021811221.nc',
 'iso_series_metadata_id': '8c9e8150-3692-11e3-aa6e-0800200c9a66',
 'title': 'ABI L2 Cloud and Moisture Imagery',
 'summary': 'Single emissive band Cloud and Moisture Imagery Products are digital maps of clouds, moisture and atmospheric 

In [26]:
#Datas->Metadata xa.fecha 
#Metadata->data 
#def class:
#     data: xa.DataArray 
#    fecha_dim coso zaraza

import dataclasses as dcss
import xarray as xa

AVAIL_ORBITS = ["Polar", "Goestationary"]
AVAIL_SATS = ["GOES", "CloudSat", "Terra"]
AVAIL_INSTRUMENTS = ["ABI", "CPR", "MODIS"]

@dcss.dataclass(frozen=True, slots=True) 
class SatelliteData:
    """Defines new satellite data.""" 
    
    data: xa.Dataset
    time_start: str
    time_end: str
    product_key: str
    instrument_type: str
    platform: str
    orbit_type: str
    
    def __post_init__(self):
        if self.orbit_type not in AVAIL_ORBITS:
            raise ValueError(f"Orbit type not valid or available. Must be one of {AVAIL_ORBITS}")
        if self.platform not in AVAIL_SATS:
            raise ValueError(f"Platform name not valid or not available. Must be one of {AVAIL_SATS}")
        if self.instrument_type not in AVAIL_INSTRUMENTS:
            raise ValueError(f"Instrument not valid or not available. Must be one of {AVAIL_INSTRUMENTS}")
        
    def to_dict(self):
        return dcss.asdict(self)
        
    
    
   

In [17]:
data = SatelliteData(goes_data, "27 jun 22 12:03", "27 jun 22 12:13", "a", "ABI", "GOES", "Geostationary") 
da = data.data
da

<xarray.Dataset>
Dimensions:                                 (y: 5424, x: 5424,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates:
    t                                       datetime64[ns] 2019-01-02T19:05:5...
  * y                                       (y) float32 0.1518 ... -0.1518
  * x                                       (x) float32 -0.1518 ... 0.1518
    y_image                                 float32 0.0
    x_image                                 float32 0.0
    band_wavelength                         (band) float32 10.33
    band_id                                 (band) int32 13
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/29)
    CMI                                     (y, x) float32 ...
    DQF                                     (y, x) float32 ...
    time_bounds                             (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                  int32 -2147483647
    y_image_bounds                          (number_of_image_bounds) float32 ...
    x_image_bounds                          (number_of_image_bounds) float32 ...
    ...                                      ...
    algorithm_dynamic_input_data_container  int32 -2147483647
    percent_uncorrectable_GRB_errors        float64 0.0
    percent_uncorrectable_L0_errors         float64 0.0
    earth_sun_distance_anomaly_in_AU        float64 0.9833
    processing_parm_version_container       int32 -2147483647
    algorithm_product_version_container     int32 -2147483647
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v25, 05 July 2013)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    time_coverage_start:       2019-01-02T19:00:36.3Z
    time_coverage_end:         2019-01-02T19:11:14.1Z
    timeline_id:               ABI Mode 3
    production_data_source:    Realtime
    id:                        5c74601f-92ef-4f97-ae10-919b95261557
    _STRATOPY_:                Metadata(orbit_type='geostationary', platform=...

In [18]:
data.instrument_name

<bound method SatelliteData.instrument_name of SatelliteData(data=<xarray.Dataset>
Dimensions:                                 (y: 5424, x: 5424,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates:
    t                                       datetime64[ns] 2019-01-02T19:05:5...
  * y                                       (y) float32 0.1518 ... -0.1518
  * x                                       (x) float32 -0.1518 ... 0.1518
    y_image                                 float32 0.0
    x_image                                 float32 0.0
    band_wavelength                         (band) float32 10.33
    band_id                                 (band) int32 13
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/29)
    CMI                                     (y, x) float32 ...
    DQF           

In [19]:
data.data

<xarray.Dataset>
Dimensions:                                 (y: 5424, x: 5424,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates:
    t                                       datetime64[ns] 2019-01-02T19:05:5...
  * y                                       (y) float32 0.1518 ... -0.1518
  * x                                       (x) float32 -0.1518 ... 0.1518
    y_image                                 float32 0.0
    x_image                                 float32 0.0
    band_wavelength                         (band) float32 10.33
    band_id                                 (band) int32 13
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/29)
    CMI                                     (y, x) float32 ...
    DQF                                     (y, x) float32 ...
    time_bounds                             (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                  int32 -2147483647
    y_image_bounds                          (number_of_image_bounds) float32 ...
    x_image_bounds                          (number_of_image_bounds) float32 ...
    ...                                      ...
    algorithm_dynamic_input_data_container  int32 -2147483647
    percent_uncorrectable_GRB_errors        float64 0.0
    percent_uncorrectable_L0_errors         float64 0.0
    earth_sun_distance_anomaly_in_AU        float64 0.9833
    processing_parm_version_container       int32 -2147483647
    algorithm_product_version_container     int32 -2147483647
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v25, 05 July 2013)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    time_coverage_start:       2019-01-02T19:00:36.3Z
    time_coverage_end:         2019-01-02T19:11:14.1Z
    timeline_id:               ABI Mode 3
    production_data_source:    Realtime
    id:                        5c74601f-92ef-4f97-ae10-919b95261557
    _STRATOPY_:                Metadata(orbit_type='geostationary', platform=...

In [27]:
ds = xa.DataArray({"coso": "cosa"})

In [28]:
data = SatelliteData(ds, "27 jun 22 12:03", "27 jun 22 12:13", "a", "ABI", "GOES", "Geostationary") 

In [29]:
data.time_coverage()
data.instrument_name()

In [30]:
data.data

<xarray.DataArray ()>
array({'coso': 'cosa'}, dtype=object)
Attributes:
    time_start:  a
    time_end:    27 jun 22 12:13
    instrument:  ABI